In [1]:
from __future__ import unicode_literals, print_function, division
import pickle as pkl
from io import open
import unicodedata
import string
import re
import random
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np, pandas as pd


In [2]:
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
teacher_forcing_ratio = 0
import math

In [3]:
from sacrebleu import raw_corpus_bleu, corpus_bleu

In [4]:
def mask_ind(arr):
    arr = arr.cpu().numpy()
    batch_size = arr.shape[1]

    for i in range(batch_size):
        if 1 in arr[:,i]:
            ind = np.where(arr[:,i]== 1)[0][0]
        
            arr[:,i][:ind+1]=1
            arr[:,i][ind+1:]=0
        else:
            arr[:,i]=1
        
    
    return arr, np.count_nonzero(arr)
                

In [5]:
def convert_idx_2_sent_new(idx_tensor, lang_obj):
    word_list = []
    #truth_word_list = []
    for i in idx_tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
#     for j in truth_tensor:
#         if j.item() not in set([PAD_IDX,EOS_token,SOS_token]):
#             truth_word_list.append(lang_obj.index2word[j.item()])
    sent = (' ').join(word_list)
    #truth_sent = (' ').join(truth_word_list)
    return sent


In [6]:
def bleu_new(corpus,truths):
    n = len(corpus)
    #bleu = [0]*n
    pred_ls = []
    true_ls = []
    for i in range(n):
        pred, true = corpus[i], truths[i]
        pred_ls.append( [convert_idx_2_sent_new(sent, target_tra) for sent in pred])
        true_ls.append([convert_idx_2_sent_new(sent, target_tra) for sent in true])
    flattened_pred  = [val for sublist in pred_ls for val in sublist]
    flattened_true  = [val for sublist in true_ls for val in sublist]
    bleu= corpus_bleu(flattened_pred, [flattened_true]).score
    return bleu

    #n = len(corpus)
    #bleu = [0]*n
    #for i in range(n):
        #pred, true = corpus[i], truths[i]
        #pred_ls = [convert_idx_2_sent_new(sent, target_tra) for sent in pred]
        #true_ls = [convert_idx_2_sent_new(sent, target_tra) for sent in true]
        #bleu[i] = corpus_bleu(pred_ls, [true_ls]).score
    #return np.mean(bleu)

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [8]:
device

'cuda'

### Data Pre-processing

In [9]:
SOS_token = 0
EOS_token = 1
PAD_IDX = 2
UNK_IDX = 3
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"PAD", 3:"UNK"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
def normalizeString(s):
#     s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"&apos;m", r"am", s)
    s = re.sub(r"&apos;s", r"is", s)
    s = re.sub(r"&apos;re", r"are", s)
    s = re.sub(r"&apos;", r"", s)
    return s

In [11]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [12]:
def loadingLangs(sourcelang, targetlang, setname):
    input_ls = []
    output_ls = []
    print('Reading lines...')
    # Read the file 
    with open('../iwslt-%s-%s/%s.tok.%s'%(sourcelang, targetlang, setname,sourcelang)) as f:
        for line in f.readlines():
            input_ls.append([normalizeString(word) for word in line.split()])
    with open('../iwslt-%s-%s/%s.tok.%s'%(sourcelang, targetlang, setname,targetlang)) as f:
        for line in f.readlines():
            output_ls.append([normalizeString(word) for word in line.split()])
    pairs = list(zip(input_ls, output_ls))
    print('Read %s sentence pairs'%(len(input_ls)))
    input_lang = Lang(sourcelang)
    output_lang = Lang(targetlang)
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [13]:
source_tra, target_tra, pairs_tra = loadingLangs('zh', 'en', 'train')
source_val, target_val, pairs_val = loadingLangs('zh', 'en', 'dev')
source_tes, target_tes, pairs_tes = loadingLangs('zh', 'en', 'test')

Reading lines...
Read 213377 sentence pairs
Counting words...
Counted words:
zh 88918
en 69063
Reading lines...
Read 1261 sentence pairs
Counting words...
Counted words:
zh 6133
en 4015
Reading lines...
Read 1397 sentence pairs
Counting words...
Counted words:
zh 5215
en 3518


### Dataset

In [14]:
MAX_SENT_LEN = 38
BATCH_SIZE = 64

In [15]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else UNK_IDX for word in sentence]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair,source,target):
    input_lang = source
    output_lang = target
    input_tensor = tensorFromSentence(input_lang, pair[0]).reshape((-1))
    target_tensor = tensorFromSentence(output_lang, pair[1]).reshape((-1))
    return (input_tensor, input_tensor.shape[0], target_tensor, target_tensor.shape[0])


In [16]:
class NMTDataset(Dataset):
    def __init__(self, source, target, pairs):
        self.source = source
        self.target = target
        self.pairs = pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        inp_ten, inp_len, tar_ten, tar_len = tensorsFromPair(self.pairs[key], self.source, self.target)
        item = {}
        item['inputtensor'] = inp_ten[:MAX_SENT_LEN]
        item['inputlen'] = min(inp_len, MAX_SENT_LEN)
        item['targettensor'] = tar_ten[:MAX_SENT_LEN]
        item['targetlen'] = min(tar_len, MAX_SENT_LEN)
        return item

In [ ]:
train_data = NMTDataset(source_tra, target_tra, pairs_tra)
val_data = NMTDataset(source_tra, target_tra, pairs_val)
test_data = NMTDataset(source_tra, target_tra, pairs_tes)

In [ ]:
train_data.__getitem__(234)

{'inputtensor': tensor([  49,  871,   16, 1235,  454, 1112,    6,   84,   85,  322,  398,  310,
            6, 1236, 1237,  735,   57, 1238,  391,  621,  611,  612,  613,   84,
           85,   16, 1239,   18,  885,    6, 1240,    1], device='cuda:0'),
 'inputlen': 32,
 'targettensor': tensor([ 48,  89,  52,  53, 577, 206,  77,  30, 113,  25,  54,  21, 210, 831,
          21,  22,  23,  56,  77, 206,  52,  53, 921, 915,  44,   1],
        device='cuda:0'),
 'targetlen': 26}

### Dataloader

In [ ]:
#collate function

def collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    src_data, tar_data, src_len, tar_len = [], [], [], []
    for datum in batch:        
        src_datum = np.pad(np.array(datum['inputtensor']),
                                pad_width=((0,MAX_SENT_LEN-datum['inputlen'])),
                                mode="constant", constant_values=PAD_IDX)
        tar_datum = np.pad(np.array(datum['targettensor']),
                                pad_width=((0,MAX_SENT_LEN-datum['targetlen'])),
                                mode="constant", constant_values=PAD_IDX)
        src_data.append(src_datum)
        tar_data.append(tar_datum)
        src_len.append(datum['inputlen'])
        tar_len.append(datum['targetlen'])
    return [torch.from_numpy(np.array(src_data)).to(device),torch.from_numpy(np.array(tar_data)).to(device),
               torch.from_numpy(np.array(src_len)).to(device),torch.from_numpy(np.array(tar_len)).to(device)]

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=BATCH_SIZE,shuffle=True,collate_fn=collate_func)

#Here change to shuffle=True
val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_func)


In [ ]:
# sample data loader
count = 0
for data in train_loader:
    count+=1
    print('input sentence tensor batch: ')
    print(data[0][3])
    print('input sentence token batch: ')
    
    print('input batch dimension: {}'.format(data[0].size()))
    print('target sentence batch: ')
    print(data[1][3])
    tokens = convert_idx_2_sent_new(data[1][3], target_tra)
    print(tokens)
    
    print('target batch dimension: {}'.format(data[1].size()))
    print('input sentence len: ')
    print(data[2])
    print('target sentence len: ')
    print(data[3])
    if count == 1:
        break

input sentence tensor batch: 
tensor([   16,   240,   404,  1212, 30553,  1095,     1,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2],
       device='cuda:0')
input sentence token batch: 
input batch dimension: torch.Size([64, 38])
target sentence batch: 
tensor([ 61, 306, 408, 751,  44,   1,   2,   2,   2,   2,   2,   2,   2,   2,
          2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
          2,   2,   2,   2,   2,   2,   2,   2,   2,   2], device='cuda:0')
We can do everything  .
target batch dimension: torch.Size([64, 38])
input sentence len: 
tensor([31, 38, 28,  7,  4, 38, 24, 31,  9, 38,  8,  9,  3,  6, 23, 14,  7,  6,
        38, 20, 13, 15, 19, 17, 14, 22, 20, 13, 28, 32,  8,  9, 23, 17,  6, 11,
        10, 23, 36, 34,  8, 10,  7, 21, 22, 38, 14, 13, 17, 14, 23, 

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx =PAD_IDX)
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True)
        self.fc1 = nn.Linear(2*hidden_size, hidden_size)
        
    def forward(self, input, hidden):
        batch_size = input.size()[1]
        #print('in Encoder, batch_size is {} \n'.format(batch_size))
        seq_len = input.size()[0]
        #print('in Encoder, seq_len is {} \n'.format(seq_len))
        embedded = self.embedding(input).view(seq_len, batch_size, -1)
        #print('in Encoder, embedded is {}, dimension is {} \n'.format(embedded, embedded.size()))
        output = embedded
        
        output, hidden = self.gru(output, hidden)
        #print('in Encoder, output after gru is {}, dimension is {} \n'.format(output, output.size()))
        #print('in Encoder, hidden after gru is {}, dimension is {} \n'.format(hidden, hidden.size()))
        
        context = self.fc1(torch.cat((hidden[0],hidden[1]),dim = 1 )).unsqueeze(0)
        #print('in encoder, context is {}, dimension is {}'.format(context, context.size()))
        #output = self.fc1(output)
        return output, context

    def initHidden(self, batch_size):
        initHidden = torch.zeros(2, batch_size, self.hidden_size, device=device)
        #print('in Encoder, initialized hidden dimension is {}'.format(initHidden.size()))
        return initHidden


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size, padding_idx = PAD_IDX)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input = input.view(1,-1)
        batch_size = input.size()[1]
        #print('in Decoder, batch_size is {}'.format(batch_size))
        
        #print('in Decoder, input before embedded layer is {}, dimension is {}'.format(input,input.size()))
        output = self.embedding(input).view(1, batch_size, -1)
        #print('in Decoder, output after embedded is {}, dimension is {} \n'.format(output, output.size()))
        output = F.relu(output)
        #print('in Decoder, output after relu is {}, dimension is {} \n'.format(output, output.size()))
        #print('in Decoder, the initial hidden is {}, dimension is {}'.format(hidden, hidden.size()))
        
        output, hidden = self.gru(output, hidden)
        #print('in Decoder, output of GRU is {}, dimension is {}'.format(output, output.size()))
        #print('in Decoder, hidden of GRU is {}, dimension is {}'.format(hidden, hidden.size()))
        
        output = self.softmax(self.out(output[0]))
        #print('in Decoder, output after softmax is {}, dimension is {}'.format(output, output.size()))
        return output, hidden

    def initHidden(self, batch_size):
        initHidden = torch.zeros(1, batch_size, self.hidden_size, device=device)
        #print('in Decoder, initHidden is {}, dimension is {} \n'.format(initHidden, initHidden.size()))
        return initHidden

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer):
    
    batch_size = input_tensor.size()[1]
    #print('in train, batch size is {}'.format(batch_size))
    encoder_hidden = encoder.initHidden(batch_size)
    #print('in train, initial encoder hidden is {}, dimension is {}'.format(encoder_hidden, encoder_hidden.size()))
    
    encoder_optimizer.zero_grad()  
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size()[0]
    #print('in train, input_length is {}'.format(input_length))
    target_length = target_tensor.size()[0]
    #print('in train, target_length is {}'.format(target_length))
    #encoder_outputs = torch.zeros(target_length, batch_size, encoder.hidden_size, device=device) 

  

    _, context = encoder(input_tensor, encoder_hidden)
    #print('in train encoder_hidden[0] is {}, dimension is {}'.format(encoder_hidden[0],encoder_hidden[0].size()))
    #print('in train after concatenating encoder_hidden[0] and [1] is {}, dimension is {}'.format(torch.cat((encoder_hidden[0].cpu().data,encoder_hidden[1].cpu().data),dim = 1), torch.cat((encoder_hidden[0].cpu().data,encoder_hidden[1].cpu().data),dim = 1).size()))
    
    #decoder_hidden = nn.Linear(2*hidden_size,hidden_size)(
        #torch.cat((encoder_hidden[0].cpu().data,encoder_hidden[1].cpu().data),dim = 1)).to(device).unsqueeze(0)
    
    decoder_input = torch.tensor([[SOS_token]*batch_size], device=device)  # decoder_input: torch.Size([1, 32])
    decoder_hidden = context.to(device)
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    #print('target_tensor is {}, dimension is {}'.format(target_tensor, target_tensor.size()))
    
    
    #print('target_tensor is {}, dimension is {}'.format(target_tensor, target_tensor.size()))
    #print('sentence 3 in this batch is {}, dimension is {}'.format(convert_idx_2_sent_new(target_tensor[:,2], target_tra)))
    #print('sentence 3 in this batch is {}'.format(convert_idx_2_sent_new(target_tensor[:,2], target_tra)))
    
    if use_teacher_forcing:
        loss = 0 
        criterion = nn.NLLLoss(reduce = True, ignore_index = 2, reduction = 'mean') 

        for di in range(target_length):
            #print('in teacher_forcing, step {}'.format(di))
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            decoder_input = target_tensor[di]  
            #print('in teacher forcing, decoder_output at current timestep is {}, dimension is {}'.format(decoder_output, decoder_output.size()))
            #print('predicted target at current timestep is {}, dimension is {}'.format(torch.argmax(decoder_output, dim=1), torch.argmax(decoder_output, dim=1).size()))
            #print('true target at current timestep is {}, dimension is {}'.format(target_tensor[i], target_tensor[i].size()))
            #print('predicted target at current timestep is {}, dimension is {}'.format(decoder_output, decoder_output.size()))
            
            temp_loss = criterion(decoder_output, target_tensor[di])
            #print ('in teacher forcing, temp loss at current step is {}'.format(temp_loss))
            #print('temp_loss for current batch, current token is {}, dimension is {}'.format(temp_loss, temp_loss.size()))
            
            loss += temp_loss
            #loss += temp_loss * mask[di:di+1].float()  
            #print('loss is {}, dimension is {}'.format(loss, loss.size()))
            #ave_loss = loss.sum()/batch_size
        ave_loss = loss/target_length
            
    else:
        loss = None 
        criterion = nn.NLLLoss(reduce = False) 
        prediction = None

        for di in range(target_length):
            #print('in non_teacher forcing, step {}'.format(di))
            
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            #print('in non_teacher forcing, topi is {}, dimension is {}'.format(topi, topi.size()))
            
            if prediction is None:
                prediction = topi.view(1,-1)
            else:
                prediction = torch.cat((prediction, topi.view(1,-1)), dim=0)
            
            #print('at current step, cumulative prediction is {}, dimension is {}'.format(prediction, prediction.size()))
            
                            
            decoder_input = topi.transpose(0,1).detach()  # detach from history as input
            #print('in non_teacher forcing, input of the current step is {}, dimension is {}'.format(topi.transpose(0,1),topi.transpose(0,1).size()))
            #print('in non_teacher forcing decoder_output at current timestep is {}, dimension is {}'.format(decoder_output, decoder_output.size()))
            
            #print('predicted target at current timestep is {}, dimension is {}'.format(torch.argmax(decoder_output, dim=1), torch.argmax(decoder_output, dim=1).size()))

            #print('true target at current timestep is {}, dimension is {}'.format(target_tensor[i], target_tensor[i].size()))
            
            temp_loss = criterion(decoder_output, target_tensor[di])
            if loss is None:
                loss = temp_loss.view(1,-1)
            else:
                loss = torch.cat((loss, temp_loss.view(1,-1)),dim=0)
            #print('temp_loss for current batch, current token is {}, dimension is {}'.format(temp_loss, temp_loss.size()))
            
            
    
    #print('Final prediction is {}'.format(prediction))
        mask, count = mask_ind(prediction)
        total_loss = torch.sum(loss * torch.from_numpy(mask).float().to(device))
        ave_loss = total_loss/count
    #print('total_loss is {}, dimension is{}'.format(total_loss, total_loss.size()))        
    ave_loss.backward()
    
    
    encoder_optimizer.step()   
    decoder_optimizer.step()
    
    #print('total valid predicted token is {}'.format(count))
    #print('ave_loss type is {}'.format(type(ave_loss)))
    #print('ave_loss.item() type is {}'.format(type(ave_loss.item())))
    
    return ave_loss.item()

In [ ]:
def evaluate(encoder, decoder, data_loader, max_length=MAX_SENT_LEN):
    start = time.time()
    encoder.eval()
    decoder.eval()
    inputs = []
    corpus = []
    truths = []
    for i, (input_sentences, target_sentences,len1,len2) in enumerate(data_loader):
#         if i % 5 == 0:
#             print('Time: {}, Step: [{}/{}]'.format(
#                 timeSince(start, i + 1/len(train_loader)), i, len(data_loader)))
        inputs.append(input_sentences.to(device))#put into inputs: batch*seq: each row is a sentence
        input_tensor = input_sentences.transpose(0,1).to(device)
        truths.append(target_sentences.to(device))#put into truths: batch*seq: each row is a sentence
        target_tensor = target_sentences.transpose(0,1).to(device) 
        #truths.append(target_tensor)
        input_length = input_tensor.size()[0]
        batch_size = input_tensor.size()[1]
    
        
        encoder_hidden = encoder.initHidden(batch_size)
        #encoder_outputs = torch.zeros(max_length, batch_size, encoder.hidden_size, device=device)
        _, context = encoder(input_tensor, encoder_hidden)
        
        #encoder_hidden = nn.Linear(2*hidden_size,hidden_size)(
        #torch.cat((encoder_hidden[0].cpu().data,encoder_hidden[1].cpu().data),dim = 1)).to(device).unsqueeze(0)
        
        decoder_hidden = context.to(device)
        decoder_input = torch.tensor([[SOS_token]*batch_size], device=device) 
        decoded_words = torch.zeros(batch_size, max_length)
    
    

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            decoded_words[:,di] = topi.squeeze()  #put into decoded_words: batch*seq
            decoder_input = topi.transpose(0,1).detach()
            #print('true target is {}, dimension is {}'.format(target_tensor[:,di],target_tensor[di].size()))
            #print('before transpose, topi is {}, dimension is {}'.format(topi, topi.size()))
            #print('after transpose, topi is {}, dimension is {}'.format(topi.transpose(0,1),topi.transpose(0,1).size()))
        corpus.append(decoded_words)
        
        #print('last: decoded_words is {}, dimension is {}'.format(decoded_words, decoded_words.size()))
        #print('last: inputs is {}, dimension is {}'.format(inputs, len(inputs)))
        #print('last: truths is {}, dimension is {}'.format(truths, len(truths)))
        #print(inputs[0].size(), corpus[0].size(), truths[0].size())
    return inputs, corpus, truths



In [ ]:
hidden_size = 256
learning_rate=0.0001
num_epoch = 10
print_every = 20
plot_every = 20

encoder1 = EncoderRNN(source_tra.n_words,hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, target_tra.n_words).to(device)

start = time.time()

encoder_optimizer = optim.Adam(encoder1.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)

for epoch in range(1, num_epoch + 1):
    plot_bleu_score_val = []

    plot_losses = []
    print_loss_total = 0  
    plot_loss_total = 0  
    for i, (input_sentences, target_sentences,len1,len2) in enumerate(train_loader): 
#         print(i)
        encoder1.train()
        decoder1.train()
        input_tensor = input_sentences.transpose(0,1)   
        target_tensor = target_sentences.transpose(0,1)
        loss = train(input_tensor, target_tensor, encoder1,
                     decoder1, encoder_optimizer, decoder_optimizer)
        print_loss_total += loss
        plot_loss_total += loss
        
        if i > 0 and i % print_every == 0:
            inputs, corpus, truths = evaluate(encoder1, decoder1, val_loader, max_length=MAX_SENT_LEN)
            bleu_score_val_avg = bleu_new(corpus, truths)#np.mean(bleu_score_val)

            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('Time: {}, Epoch: [{}/{}], Step: [{}/{}], Train Loss: {}, BLEU: {}'.format(
                timeSince(start, i + 1/len(train_loader)), epoch, num_epoch, i, 
                len(train_loader),print_loss_avg,bleu_score_val_avg))
            
            print('\nInput1:> %s'%(' '.join([source_tra.index2word[i.item()] for i in inputs[0][3] if i.item() not in set([PAD_IDX,EOS_token,SOS_token])])))
            print('\nTarget1:= %s'%(convert_idx_2_sent_new(truths[0][3], target_tra)),
                    '\nPredict1:< %s' %(convert_idx_2_sent_new(corpus[0][3], target_tra)))
            
            print('\nInput2:> %s'%(' '.join([source_tra.index2word[i.item()] for i in inputs[1][3] if i.item() not in set([PAD_IDX,EOS_token,SOS_token])])))
            print('\nTarget2:= %s'%(convert_idx_2_sent_new(truths[1][3], target_tra)),
                    '\nPredict2:< %s' %(convert_idx_2_sent_new(corpus[1][3], target_tra)))

        if i > 0 and i % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
    print(plot_losses)

        #if i % print_every == 0:
            #corpus, truths = evaluate_rnn(encoder1, attn_decoder1, val_loader, max_length=MAX_SENT_LEN)
            #score_ls = bleu(corpus, truths)
            #avg_score = np.array(score_ls).mean()
            #print_loss_avg = print_loss_total / print_every
            #print_loss_total = 0
            #print('Time: {}, Epoch: [{}/{}], Step: [{}/{}], Train Loss: {}, Average BLEU: {}'.format(
                    #timeSince(start, i + 1/len(train_loader)), epoch, num_epoch, i, 
                    #len(train_loader),print_loss_avg, avg_score))

        #if i > 0 and i % plot_every == 0:
            #plot_loss_avg = plot_loss_total / plot_every
            #plot_losses.append(plot_loss_avg)
            #plot_loss_total = 0

                
    #print(plot_losses)
        


/home/zh1087/nlp_environment/py3.6.3/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Time: 0m 5s (- -1m 54s), Epoch: [1/10], Step: [20/3335], Train Loss: 11.516338539123534, BLEU: 0.00020406701288841283

Input1:> 如果 想见 我 就 拿 着 UNK 来 花园 吧 这样 我们 就 可以 种 点 东西 啦

Target1:= If you want to meet with me , come to the garden with your shovel so we can plant some shit  . 
Predict1:< defecate defecate defecate defecate defecate

Input2:> 但是 要 尝试 试一下 一下

Target2:= But I will do a trial  . 
Predict2:< Corfu defecate defecate defecate defecate
Time: 0m 11s (- -1m 49s), Epoch: [1/10], Step: [40/3335], Train Loss: 10.959937858581544, BLEU: 0.0

Input1:> 所以 我 希望 分享 这种 主动 且 有意 有意识 意识 去 创造 记忆 的 想法

Target1:= So I want to share the idea of taking an active role in consciously creating memories  . 
Predict1:< 

Input2:> 因此 我 今天 站 在 这里 流 着 阿富汗 的 血 以及 上帝 授予 我 的 乡土 作风 一个 无神 无神论 无神论者 神论 神论者 论者 和 一个 激进 的 政治 艺术 艺术家 术家 并且 在 过去 九年 一直

Target2:= And so I stand here today , Afghan by blood , redneck by the grace of God , an atheist and a radically politicized artist who is been living , working and 

Time: 0m 27s (- -1m 33s), Epoch: [1/10], Step: [100/3335], Train Loss: 5.304116988182068, BLEU: 0.019996369928600218

Input1:> UNK 会议 主持 总统 他们 没有 创造 这个 个头 头衔 我 的 意思 是 这个 个头 头衔 以前 就 有 但是 它 是 指 一个 会议 的 主持 主持人

Target1:= President  . They didn t invent the title  . I mean , it existed before , but it just meant somebody who UNK over a meeting  . 
Predict1:< And  .  .  .  .  .

Input2:> 所以 从今 今天 天开 开始 朝 这个 方向 迈出 第一 第一步 一步 我们 开始 了 UNK s Voice UNK 这个 项目

Target2:= So , taking the first steps towards this today , we are launching the Parkinson is Voice Initiative  . 
Predict2:< And  .  .  .  .  .
Time: 0m 32s (- -1m 28s), Epoch: [1/10], Step: [120/3335], Train Loss: 4.272518944740296, BLEU: 0.0035138235591212997

Input1:> 我 见到 他 时 他 已 在 矿场 工作 了 14 年 照片 片中 看到 的 这个 腿伤 是 在 一次 凿 矿 意外 中 造成 的 当时 情况 UNK 危急 医生 说 要 截肢

Target1:= When I met him , he had been working in the mines for 14 years , and the leg injury that you see here is actually from a mining accident , one so severe doctors say his leg 
P

Time: 1m 50s (- -2m 9s), Epoch: [1/10], Step: [420/3335], Train Loss: 3.513837218284607, BLEU: 0.05556175303154072

Input1:> 我们 必须 学会 如何 让 这些 人 给出 他们 的 意见

Target1:= You have to learn how to get these people to come and talk to you  . 
Predict1:< And is is is the  .  .

Input2:> 举例 举例来说 来说 一直 直到 UNK 年 英国 英国议会 国议会 议会 才 允许 新闻 闻报 报纸 报道 在 辩论 辩论会 会上 政客 们 说 的 真实 实话 话语

Target2:= It wasn t until , for example , UNK that the British Parliament allowed newspapers to report the exact words that were said in the debating chamber  . 
Predict2:< And , , , , , , , , , , , , , , , , , , , , , , ,


Time: 1m 55s (- -2m 4s), Epoch: [1/10], Step: [440/3335], Train Loss: 3.519381070137024, BLEU: 0.03534524268133123

Input1:> 他 告诉 我 两件 两件事

Target1:= He told me two things  . 
Predict1:< And is is , , ,

Input2:> 别误会 误会

Target2:= Now don t get me wrong  . 
Predict2:< And is is is


...

Time: 94m 1s (- -94m 9s), Epoch: [7/10], Step: [500/3335], Train Loss: 3.379266107082367, BLEU: 0.8005072867208092

Input1:> 比如 艾哈迈 艾哈迈德 哈迈德 迈德 这 不是 真名 我 也 不能 让 你们 看见 他 的 长相 他 是 我 一个 学生 的 父亲

Target1:= Like Ahmed  . That is not his real name , and I cannot show you his face , but Ahmed is the father of one of my students  . 
Predict1:< For , , , , , , , , , , , , , , , , , , , , , , ,  .

Input2:> 在 这个 物欲 UNK 横流 的 时代 里 努力 坚持 自我 铭记 人生 的 短暂 与 生命 的 脆弱 变得 比 以往 任何 何时 时代 都 更 重要

Target2:= In our age of increasing distractions , it is more important than ever to find ways to maintain perspective and remember that life is brief and tender  . 
Predict2:< In the , , , , , , , , , , , , , , , , , , , , ,


Time: 94m 7s (- -94m 3s), Epoch: [7/10], Step: [520/3335], Train Loss: 3.30777622461319, BLEU: 0.8159498173566931

Input1:> 这 让 我们 相信 自己 并 不 孤单

Target1:= It is about knowing you are not alone  . 
Predict1:< It is not us us to  .  .  .

Input2:> 它 随处 随处可见 可见

Target2:= It is everywhere  . 
Predict2:< It is it  .  .


Time: 94m 12s (- -95m 57s), Epoch: [7/10], Step: [540/3335], Train Loss: 3.275642716884613, BLEU: 0.7906745039700658

Input1:> 鼓掌 基本 基本上 如果 你们 想见 我 如果 想要 碰面 如果 想 坐在 舒适 的 椅子 上 讲 一些 有 的 没 的 就 别 给 我 打电话 电话

Target1:= So basically , if you want to meet with me , you know , if you want to meet , don t call me if you want to sit around in cushy chairs and have meetings where you 
Predict1:< And , I I I I if I if I if I if I , , , , , , , , , , , , , ,

Input2:> 我 曾经 误以为 以为 只有 我 一个 个人 有 这样 的 遭遇

Target2:= I was mistaken in thinking that I was unique and alone in this situation  . 
Predict2:< I I a a a , , , I  .  .  .


Time: 94m 18s (- -95m 51s), Epoch: [7/10], Step: [560/3335], Train Loss: 3.3543291091918945, BLEU: 0.8072013383711368

Input1:> 那些 枪 对 我 来说 是 一个 明显 的 信号 尽管 他 并 没有 拿起 枪 指着 我 我 已经 无时 无时不刻 不 处在 危险 的 边缘

Target1:= But those guns were really a message for me , and even though he hadn t raised a hand to me , my life was already in grave danger every minute of every day  . 
Predict1:< And is is , , , , , , , , , , , , , , , , , , , , , , , , ,

Input2:> 我们 能 减少 和 病人 有关 的 后勤 问题

Target2:= We can reduce logistical difficulties with patients  . 
Predict2:< We can we the the the  .  .


Time: 94m 23s (- -95m 45s), Epoch: [7/10], Step: [580/3335], Train Loss: 3.263369154930115, BLEU: 0.8118237030883791

Input1:> 由此 开始 了 我 深入 现代 奴隶 奴隶制 奴隶制度 制度 的 旅程

Target1:= Thus began my journey into modern day slavery  . 
Predict1:< And then I , I I , , the  .  .  .

Input2:> 所以 现在 的 沥青 有 一些 些小 钢丝 丝绒 在 里面

Target2:= So then you have asphalt with very small pieces of steel wool in it  . 
Predict2:< So now are are are are the  .  .  .
